In [ ]:
import os

from mpl_toolkits.mplot3d import Axes3D
from matplotlib.pyplot import imshow, figure, bar, hist, show

from skimage import io, exposure
from skimage.measure import compare_ssim as ssim
from skimage.transform import resize
from skimage.util import img_as_ubyte
from skimage.filters import sobel

import imagehash
from PIL import Image
import numpy as np
from shutil import copyfile

import plotly.plotly as py
from plotly.graph_objs import *

In [ ]:
DebugImage = False

PATH_TO_TRAIN = "../../data/input/train"
PATH_TO_SAVE_NORMALIZED_IMAGES = "../../data/segmentation.normalization"

In [ ]:
images_to_process = []

for current_directory in os.listdir(PATH_TO_TRAIN):
    current_image = {}
    current_image["id"] = current_directory
    current_image["images"] = []
    current_image["masks"] = []
    for current_image_file in os.listdir(os.path.join(PATH_TO_TRAIN,current_directory,"images")):
        current_image["images"].append(os.path.join(PATH_TO_TRAIN,current_directory,"images", current_image_file))
    
    for current_image_mask in os.listdir(os.path.join(PATH_TO_TRAIN,current_directory,"masks")):
        current_image["masks"].append(os.path.join(PATH_TO_TRAIN,current_directory,"masks", current_image_mask))
   
    images_to_process.append(current_image)

In [ ]:
layout = Layout(
    title='Parametric Plot',
    scene=Scene(
        xaxis=XAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        yaxis=YAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        ),
        zaxis=ZAxis(
            gridcolor='rgb(255, 255, 255)',
            zerolinecolor='rgb(255, 255, 255)',
            showbackground=True,
            backgroundcolor='rgb(230, 230,230)'
        )
    )
)

In [ ]:
max_width = 0
max_height = 0
size = 0

for current_image in images_to_process:
    image_file = current_image['images'][0]
    file_png = io.imread(image_file, as_grey=True) 
    current_shape = file_png.shape
    
    if current_shape[0] > max_width :
        max_width = current_shape[0]
    if current_shape[1] > max_height :
        max_height = current_shape[1]
        
print('max_width:' , max_width, ', max_height:', max_height)

if max_width > max_height:
    size = max_width
else:
    size = max_height
    
size= int(size / 256) * 256    
    
print('image final size is ', size)    

In [ ]:
%matplotlib inline


def generate_augmentation(file_name, image, mask):
    current_image = image
    current_mask = mask
    for current_rotation in range(4):
        if current_rotation > 0:
            for current_angle in range(current_rotation):
                current_image = np.rot90(current_image)
                current_mask = np.rot90(current_mask)
        io.imsave(os.path.join(PATH_TO_SAVE_NORMALIZED_IMAGES,'{}.{}'.format(current_rotation,file_name)),current_image)
        io.imsave(os.path.join(PATH_TO_SAVE_NORMALIZED_IMAGES,'{}.{}'.format(current_rotation,file_name)).replace('.png','.mask.png'),current_mask)
        current_image = np.bitwise_and(current_image,current_mask)
        io.imsave(os.path.join(PATH_TO_SAVE_NORMALIZED_IMAGES,'{}.{}'.format(current_rotation,file_name)).replace('.png','.mask.values.png'),current_image)
        
        
total_files = len(images_to_process)
counter = 0
for current_image in images_to_process:
    
    image_file = current_image['images'][0]
    image_name = image_file.split("/")[-1:][0]
    print("Processing:{}% - {}".format(int((counter / total_files) * 100), image_name))
    
    file_png = io.imread(image_file, as_grey=True) 
    file_png = resize(file_png , (size,size))

    if DebugImage:
        file_png = file_png[1:20,1:20]
       
    file_png = img_as_ubyte(file_png)
    #print(file_png.mean())
    if file_png.mean() > 64: 
        #print("Inverting the color of the image")
        file_png = np.invert(file_png)

    array1 = np.asarray(file_png)
    min_value = array1.min()
    max_value = array1.max()
    current_range = max_value - min_value
    
    #Original Image
    if DebugImage:
        print(array1.shape, min_value,max_value, current_range)
        figure(figsize=(8, 6), dpi=80)
        imshow(array1, cmap='gray',vmin=0,vmax=255)

    #Normalized Image
    file_png = file_png - min_value
    file_png = file_png * (255 / current_range )
    array1 = np.asarray(file_png)
    min_value = array1.min()
    max_value = array1.max()
    current_range = max_value - min_value
    if DebugImage:
        print(array1.shape, min_value,max_value, current_range)
        figure(figsize=(8, 6), dpi=80)
        imshow(array1, cmap='gray',vmin=0,vmax=255)

    #Surface Plot
    if DebugImage:
        fig = figure(figsize=(8, 6), dpi=80)
        ax = fig.gca(projection='3d')
        X, Y = np.meshgrid(np.arange(0, array1.shape[0]) ,  np.arange(0,array1.shape[1]))
        Z = np.array(array1[X, Y])
        surf = ax.plot_surface(X , Y, Z)
        show()
        
    array1 = array1.astype('uint8')

    current_mean = array1.mean()
    array1[file_png < (current_mean / 2)] = 0
    
    masked_img = None
    first = True
    for mask in current_image['masks']:
        cur_img = io.imread(mask, as_grey=True)
        cur_img = resize(cur_img , (size,size))

        cur_img = img_as_ubyte(cur_img)
        cur_img[cur_img > 0] = 255
        
        cur_img = sobel(cur_img)
        
        cur_img = img_as_ubyte(cur_img)
        cur_img[cur_img > 0] = 255
        
        if first:
            masked_img = cur_img
        else:
            masked_img = np.bitwise_or(masked_img,cur_img)
        first = False 
    
    print("Valores:{} img={}-{} mask={}-{} ".format(array1.shape, array1.min(), array1.max(), masked_img.min(), masked_img.max()))
    generate_augmentation(image_name,array1,masked_img)
    counter+=1

    
